In [1]:
!mkdir hgm_pb_only -p
import os
os.chdir('hgm_pb_only')
from tools.procOps import *
from tools.fileOps import *
import pandas as pd
from cat.hints_db import *
from tools.misc import *
from tools.intervals import *
from tools.transcripts import *
import itertools
import re
from cat.hgm import extract_exon_hints
from collections import *
from tools.bio import *
from cat.plots import *
from tools.psl import *
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
# to run homGeneMapping, I need to use the existing hints database which contains the IsoSeq hints

# for each genome, I construct a reference GTF consisting of ab-initio exons and CDS
# I then lift them over to target genomes

# we want to use both CDS and exon intervals because CGP has a coding-only model, and so the CDS interval
# will be the exonic interval

# I also construct a supplemental GFF that consists of the transMap exons (or gencode for human)
# these will let me know which exons are novel

In [9]:
genomes = ['Human', 'Chimp', 'Gorilla', 'Orangutan', 'Rhesus']

In [3]:
gencode_gp = '/public/groups/cgl/cat/primates_evan/work/reference/gencode.v33.annotation.gff3.gp'

def gene_pred_to_gtf(tx):
    """Extract both exons and CDS intervals as CDS"""
    for exon in tx.exon_intervals:
        yield [tx.chromosome, 'CAT', 'exon', exon.start + 1, exon.stop, '.', exon.strand, '.', 'transcript_id "{}; gene_id "{}"'.format(tx.name, tx.name2)]
    cds_tx = Transcript(tx.get_bed(new_start=tx.thick_start, new_stop=tx.thick_stop))
    for exon in cds_tx.exon_intervals:
        yield [tx.chromosome, 'CAT', 'exon', exon.start + 1, exon.stop, '.', exon.strand, '.', 'transcript_id "{}_cds"; gene_id "{}"'.format(tx.name, tx.name2)]
        
def gene_pred_to_gff(tx):
    """Extract both exons and CDS intervals as CDS"""
    for exon in tx.exon_intervals:
        yield [tx.chromosome, 'CAT', 'exon', exon.start + 1, exon.stop, '.', exon.strand, '.', 'source=A']
    cds_tx = Transcript(tx.get_bed(new_start=tx.thick_start, new_stop=tx.thick_stop))
    for exon in cds_tx.exon_intervals:
        yield [tx.chromosome, 'CAT', 'exon', exon.start + 1, exon.stop, '.', exon.strand, '.', 'source=A']
        

with open('gtfs.tbl', 'w') as outf:
    for genome in genomes:
        # isolate ab-initio transcripts that are spliced
        cgp_gp = '/public/groups/cgl/cat/primates_evan/work/augustus_cgp/{}.augCGP.gp'.format(genome)
        pb_gp = '/public/groups/cgl/cat/primates_evan/work/augustus_pb/{}.augPB.gp'.format(genome)
        out_gtf = genome + '.ab_initio_exons.gtf'
        with open(out_gtf, 'w') as out_gtf_handle:
            for gp in [cgp_gp, pb_gp]:
                for tx in gene_pred_iterator(gp):
                    if len(tx.intron_intervals) > 0:
                        print_rows(out_gtf_handle, list(gene_pred_to_gtf(tx)))
        supp_gff = genome + '.extrinsic_supplement.gff'
        with open(supp_gff, 'w') as supp_gff_handle:
            # copy all transMap or GENCODE exons with source=A to the supplement
            if genome == 'Human':
                annot_gp = gencode_gp
            else:
                annot_gp = '/public/groups/cgl/cat/primates_evan/work/transMap/{}.gp'.format(genome)
            for tx in gene_pred_iterator(annot_gp):
                print_rows(supp_gff_handle, list(gene_pred_to_gff(tx)))
        print_row(outf, [genome, out_gtf, supp_gff])

In [ ]:
# I ran this on kubernetes
!homGeneMapping --halfile=primates_eichler.hal --gtfs=gtfs.tbl --cpus=32 --dbaccess=hints.db

In [7]:
# load HGM output
parsed_hgm = {}

# 0	Chimp
# 1	Gorilla
# 2	Human
# 3	Orangutan
# 4	Rhesus
r = re.compile('\**-')
species_map = {'0': 'Chimp', '1': 'Gorilla', '2': 'Human', '3': 'Orangutan', '4': 'Rhesus'}

def parse_info_line(hgm_info):
    """
    For a given info line, we want to know what genomes it was supported in, and at what level
    
    Returns a dict with key-values of PB/N/M, etc, value is multiplicity
    """
    ret_dict = {}  # keeps mapping of genome name to values
    for l in hgm_info.split(','):
        ret = {}
        if 'PB' in l or 'A' in l:  # make sure we have something to look at
            g = species_map[l[0]]
            sl = l[1:].split(':')  # strip species identifier
            for v in sl:
                if '-' in v:
                    feature_type, mult = r.split(v)
                    ret[feature_type] = int(mult)
                else:
                    ret[v] = 1
            ret_dict[g] = ret
    return ret_dict

for g in genomes:
    e = []
    with open(g + '.gtf') as infile:
        for line in infile:
            if '\texon\t' in line:
                line = line.rstrip().split('\t')
                attr_line = line[-1]
                attributes = parse_gtf_attr_line(attr_line)
                data = parse_info_line(attributes['hgm_info'])
                i = ChromosomeInterval(line[0], int(line[3]) - 1, int(line[4]), '.', data)
                e.append([i, attributes['transcript_id'].split('_')[0]])
    parsed_hgm[g] = e

In [10]:
supported_dfs = {}
features = ['PB', 'A']
cols = ['ab_initio_tx_id', 'chromosome', 'start', 'stop']
for g in genomes:
    for m in features:
        cols.append(g + '_' + m)

for genome, e in parsed_hgm.items():
    df = []  # dataframe with values chromosome, start, stop, then genome support levels in order
    for i, tx_id in e:
        # filter out poorly annotated human sequences
        if genome == 'Human' and ('alt' in i.chromosome or 'random' in i.chromosome or 'chrUn' in i.chromosome):
            continue
        r = [tx_id, i.chromosome, i.start, i.stop]  # row
        for g in genomes:
            for m in features:
                if g in i.data:
                    r.append(i.data[g].get(m, 0))
                else:
                    r.append(0)
        df.append(r)
    
    df = pd.DataFrame(df, columns=cols)
    supported_dfs[genome] = df.drop_duplicates()
            

In [11]:
# load gene info for output
annotation_maps = {}
for genome in genomes:
    if genome == 'Human':
        continue
    df = pd.read_csv(os.path.join('/public/groups/cgl/cat/primates_evan/out/consensus_gene_set', genome + '.gp_info'), sep='\t')
    df = df[['alignment_id', 'source_gene', 'source_gene_common_name']]
    df.columns = ['ab_initio_tx_id', 'gene_id', 'gene_name']
    annotation_maps[genome] = df

In [13]:
# perform parent gene assignment on Human
from cat.parent_gene_assignment import assign_parents
parents = []
for d in ['/public/groups/cgl/cat/primates_evan/work/augustus_cgp/Human.augCGP.gp', 
          '/public/groups/cgl/cat/primates_evan/work/augustus_pb/Human.augPB.gp']:
    tx_dict = get_gene_pred_dict(d)
    tx_dict = {tx_id: tx for tx_id, tx in tx_dict.items() if len(tx.exon_intervals) > 1}
    # abuse parental assignment code by passing reference as unfiltered and filtered transMap
    parents.append(assign_parents('/public/groups/cgl/cat/primates_evan/work/reference/gencode.v33.annotation.gff3.gp', 
                                  '/public/groups/cgl/cat/primates_evan/work/reference/gencode.v33.annotation.gff3.gp', 
                                  '/public/groups/cgl/cat/primates_evan/work/genome_files/Human.chrom.sizes', d))
assignment_df = pd.concat(parents)

In [14]:
from tools.sqlInterface import *
ref_df = load_annotation('/public/groups/cgl/cat/primates_evan/out/databases/Human.db')

In [18]:
human_df = assignment_df.merge(ref_df, left_on='AssignedGeneId', right_on='GeneId')
human_df = human_df[['TranscriptId', 'GeneId', 'GeneName']]
human_df.columns = ['ab_initio_tx_id', 'gene_id', 'gene_name']
annotation_maps['Human'] = human_df

In [67]:
strict_novel_exon_dfs = {}
for g, df in supported_dfs.items():
    if g == 'Human':
        continue
    subset_df = df[df['{}_PB'.format(g)] > 2]
    for gg in genomes:
        subset_df = subset_df[subset_df['{}_A'.format(gg)] == 0]
    subset_df = subset_df.merge(annotation_maps[g], on='ab_initio_tx_id')
    subset_df = subset_df[~subset_df.gene_id.isnull()]
    for gg in genomes:
        subset_df = subset_df.drop('{}_A'.format(gg), axis=1)
    # drop duplicates
    subset_df = subset_df.groupby(['chromosome', 'start', 'stop']).first().reset_index()
    subset_df.to_csv('{}.txt'.format(g), sep='\t', index=False)
    strict_novel_exon_dfs[g] = subset_df


In [298]:
# how many of the strict set have no overlap with any comparative exon, including TMR?

useful_biotypes = {'processed_pseudogene',
 'protein_coding',
 'transcribed_processed_pseudogene',
 'transcribed_unitary_pseudogene',
 'transcribed_unprocessed_pseudogene',
 'unprocessed_pseudogene'}

for genome, df in strict_novel_exon_dfs.items():
    with TemporaryDirectoryPath() as tmpdir:
        tmpdir = 'tmp'
        !mkdir -p {tmpdir}
        tmp_novel = os.path.join(tmpdir, '{}.bed'.format(genome))
        tmp_comparative = os.path.join(tmpdir, '{}.comparative.bed'.format(genome))
        tmp_comparative_bed6_sorted = os.path.join(tmpdir, '{}.comparative.bed6.bed'.format(genome))
        df[['chromosome', 'start', 'stop']].to_csv(tmp_novel, sep='\t', header=None, index=False)
        !bedSort {tmp_novel} {tmp_novel}
        if genome == 'Human':
            !cp '/public/groups/cgl/cat/primates_evan/work/reference/gencode.v33.annotation.gff3.bed' {tmp_comparative}
        else:
            # transMap doesn't keep genePred anymore
            psl = f'/public/groups/cgl/cat/primates_evan/work/transMap/{genome}.psl'
            !pslToBed {psl} {tmp_comparative}
            # adding augTMR removes too many true positives
            #with open(tmp_comparative, 'a') as fh:
            #    for f in [f'/public/groups/cgl/cat/primates_evan/work/augustus/{genome}.augTM.gp',
            #              f'/public/groups/cgl/cat/primates_evan/work/augustus/{genome}.augTMR.gp']:
            #        for rec in gene_pred_iterator(f):
            #            print_row(fh, rec.get_bed())
        !bedtools bed12tobed6 -i {tmp_comparative} | sort -k1,1 -k2,2n | uniq > {tmp_comparative_bed6_sorted}
        # find complement of intersection of flat per-exon BED with novel calls
        !bedtools intersect -v -a {tmp_novel} -b {tmp_comparative_bed6_sorted} > {genome}.no_overlap.txt
        # re-add columns
        p = genome + '.no_overlap.txt'
        tmp_df = pd.read_csv(p, sep='\t', header=None)
        tmp_df.columns = ['chromosome', 'start', 'stop']
        tmp_df = tmp_df.merge(df, on=['chromosome', 'start', 'stop'])
        tmp_df = tmp_df[~tmp_df.gene_id.isnull()]
        # add biotypes to filter out non-coding
        tmp_df = tmp_df.merge(ref_df[['GeneBiotype', 'GeneName']], left_on='gene_name', right_on='GeneName')
        # filter for known genes
        tmp_df = tmp_df[~tmp_df.GeneName.str.contains('\.[0-9]$')]
        tmp_df = tmp_df[tmp_df.GeneBiotype.isin(useful_biotypes)]
        tmp_df = tmp_df.drop(['GeneBiotype', 'GeneName'], axis=1)
        tmp_df = tmp_df.drop_duplicates()
        tmp_df.to_csv(p, sep='\t', index=False)
        tmp_df.to_excel(genome + '.no_overlap.xlsx')

***** WARNING: File tmp/Chimp.comparative.bed6.bed has inconsistent naming convention for record:
chr1	0	185	ENST00000470030.6-0	984	-

***** WARNING: File tmp/Chimp.comparative.bed6.bed has inconsistent naming convention for record:
chr1	0	185	ENST00000470030.6-0	984	-

***** WARNING: File tmp/Gorilla.comparative.bed6.bed has inconsistent naming convention for record:
chr1	1113	1127	ENST00000400809.8-0	978	-

***** WARNING: File tmp/Gorilla.comparative.bed6.bed has inconsistent naming convention for record:
chr1	1113	1127	ENST00000400809.8-0	978	-

***** WARNING: File tmp/Orangutan.comparative.bed6.bed has inconsistent naming convention for record:
chrM	8351	8660	ENST00000603319.1-2	594	+

***** WARNING: File tmp/Orangutan.comparative.bed6.bed has inconsistent naming convention for record:
chrM	8351	8660	ENST00000603319.1-2	594	+



In [213]:
# try to filter these calls by genomic alignments
tx_dict = load_gps(['/public/groups/cgl/cat/primates_evan/work/augustus_pb/Rhesus.augPB.gp',
                    '/public/groups/cgl/cat/primates_evan/work/augustus_cgp/Rhesus.augCGP.gp'])
seq_dict = get_sequence_dict('/public/groups/cgl/cat/primates_evan/work/genome_files/Rhesus.fa')


In [299]:
tmp_fa = 'tmp.fa'
tmp_psl = 'tmp.psl'
tx_ids = set(tmp_df.ab_initio_tx_id) 

with open(tmp_fa, 'w') as fh:
    for i, s in tmp_df.iterrows():
        seq = seq_dict[s.chromosome][s.start:s.stop]
        write_fasta(fh, i, seq)

In [300]:
!minimap2 -t 30 -ax sr /public/groups/cgl/cat/primates_evan/work/genome_files/Human.fa {tmp_fa} > tmp.sam

[M::mm_idx_gen::127.097*1.73] collected minimizers
[M::mm_idx_gen::139.696*2.81] sorted minimizers
[M::main::139.696*2.81] loaded/built the index for 455 target sequence(s)
[M::mm_mapopt_update::139.696*2.81] mid_occ = 1000
[M::mm_idx_stat] kmer size: 21; skip: 11; is_hpc: 0; #seq: 455
[M::mm_idx_stat::153.582*2.65] distinct minimizers: 381630162 (93.71% are singletons); average occurrences: 1.335; average spacing: 6.297
[M::worker_pipeline::153.612*2.65] mapped 437 sequences
[M::main] Version: 2.17-r941
[M::main] CMD: minimap2 -t 30 -ax sr /public/groups/cgl/cat/primates_evan/work/genome_files/Human.fa tmp.fa
[M::main] Real time: 155.673 sec; CPU: 408.642 sec; Peak RSS: 20.551 GB


In [301]:
!bamToPsl -nohead tmp.sam tmp.psl

In [302]:
recs = list(psl_iterator('tmp.psl'))

In [303]:
recs = group_alignments_by_qname(recs)

In [304]:
# find best hit
best_recs = {}
for x, y in recs.items():
    y = sorted(y, key=lambda v: -v.coverage)[0]
    best_recs[x] = y

In [305]:
# worth looking at
cov_df = pd.DataFrame([[x.q_name, x.coverage <= 0.95] for x in best_recs.values()])
cov_df.columns = ['index', 'coverage_loss']
cov_df.set_index('index')
subset_df = cov_df.merge(tmp_df, left_index=True, right_index=True)
subset_df = subset_df[subset_df.coverage_loss == True]

In [309]:
tmp_df.sample(20)

,chromosome,start,stop,ab_initio_tx_id,Human_PB,Chimp_PB,Gorilla_PB,Orangutan_PB,Rhesus_PB,gene_id,gene_name
1324,chr15,12780785,12780827,augCGP-295.t1,0,0,0,2,315,ENSG00000106991.13,ENG
4125,chr9,28961222,28961264,augPB-9895.t3,0,0,0,0,18,ENSG00000150054.18,MPP7
3103,chr6,52525883,52525952,augPB-22433.t1,0,0,0,0,11,ENSG00000185305.11,ARL15
3063,chr6,5400653,5400750,augPB-22259.t1,0,0,0,0,6,ENSG00000164151.12,ICE1
2469,chr3,42640874,42640922,augPB-21146.t2,0,0,0,0,8,ENSG00000106351.13,AGFG2
1535,chr16,32539843,32539951,augPB-14973.t1,0,0,0,0,76,ENSG00000277632.2,CCL3
3362,chr6,176515786,176515919,augPB-23268.t2,0,0,0,0,15,ENSG00000087116.16,ADAMTS2
4192,chr9,38125940,38126002,augCGP-3183.t1,0,0,0,0,38,ENSG00000175302.5,ANKRD30BP1
2012,chr2,43454034,43454095,augPB-6005.t2,0,0,0,0,3,ENSG00000155893.13,PXYLP1
163,chr1,116555600,116555693,augPB-8180.t3,0,0,0,0,5,ENSG00000134215.16,VAV3
